In [1]:
import os
import json
import random
random.seed(1)

In [2]:
with open("/data/nzxyin/GLOBE/preprocess_default/accent_clean_10.json", 'r') as f:
    filename_to_accent = json.load(f)

In [21]:
text_ids_dir = set(filename.split('.')[0] for filename in os.listdir("/data/nzxyin/GLOBE/preprocess_default/text_ids/"))
sparc_dir = set(filename.split('.')[0] for filename in os.listdir("/data/nzxyin/GLOBE/preprocess_default/sparc/"))

In [22]:
american_english = [k for k, v in filename_to_accent.items() if v == 'American English' and k.split('.')[0] in text_ids_dir and k.split('.')[0] in sparc_dir]
british_english = [k for k, v in filename_to_accent.items() if v == 'British English' and k.split('.')[0] in text_ids_dir and k.split('.')[0] in sparc_dir]

In [23]:
len(american_english), len(british_english)

(370017, 94262)

In [24]:
random.shuffle(american_english)
random.shuffle(british_english)

In [25]:
american_train, american_dev, american_test = american_english[:-10000], american_english[-10000:-5000], american_english[-5000:]

In [26]:
british_train, british_dev, british_test = british_english[:-10000], british_english[-10000:-5000], british_english[-5000:]

In [27]:
os.makedirs('/data/nzxyin/GLOBE/preprocess_default/splits/american_english_splits', exist_ok=True)
with open('/data/nzxyin/GLOBE/preprocess_default/splits/american_english_splits/train.json', 'w') as f:
    json.dump(american_train, f)
with open('/data/nzxyin/GLOBE/preprocess_default/splits/american_english_splits/val.json', 'w') as f:
    json.dump(american_dev, f)
with open('/data/nzxyin/GLOBE/preprocess_default/splits/american_english_splits/test.json', 'w') as f:
    json.dump(american_test, f)

In [28]:
os.makedirs('/data/nzxyin/GLOBE/preprocess_default/splits/british_english_splits', exist_ok=True)
with open('/data/nzxyin/GLOBE/preprocess_default/splits/british_english_splits/train.json', 'w') as f:
    json.dump(british_train, f)
with open('/data/nzxyin/GLOBE/preprocess_default/splits/british_english_splits/val.json', 'w') as f:
    json.dump(british_dev, f)
with open('/data/nzxyin/GLOBE/preprocess_default/splits/british_english_splits/test.json', 'w') as f:
    json.dump(british_test, f)

In [29]:
import numpy as np

In [20]:
dur = np.load("/data/nzxyin/GLOBE/preprocess_default/durations/S_017790_103552.npy")
art = np.load("/data/nzxyin/GLOBE/preprocess_default/sparc/S_017790_103552.npy")

FileNotFoundError: [Errno 2] No such file or directory: '/data/nzxyin/GLOBE/preprocess_default/durations/S_017790_103552.npy'

In [57]:
dur

array([ 2,  9,  4,  9, 10,  1,  5,  2,  2,  5,  3,  2,  4,  3,  2,  3,  5,
        9,  4,  2,  6,  4,  5,  6,  5,  4,  6,  7,  3,  2,  3,  3,  4,  3,
        2,  2,  3,  2,  3,  5,  3,  6,  9,  2])

In [58]:
art.shape

(185, 15)

In [59]:
sum(dur), len(art)

(np.int64(184), 185)

In [43]:
620/155

4.0

In [ ]:
durations_dir = os.listdir("/data/nzxyin/GLOBE/preprocess_default/durations/")
text_ids_dir = os.listdir("/data/nzxyin/GLOBE/preprocess_default/text_ids/")
for filename in durations_dir:
    # assert filename in sparc_dir, print(filename)
    assert filename in text_ids_dir, print(filename)

ARPA Features

In [ ]:
_front = {
    "AA": -1,
    "AA0": -1,
    "AA1": -1,
    "AA2": -1,
    "AE": 1,
    "AE0": 1,
    "AE1": 1,
    "AE2": 1,
    "AH": -1,
    "AH0": -1,
    "AH1": -1,
    "AH2": -1,
    "AO": -1,
    "AO0": -1,
    "AO1": -1,
    "AO2": -1,
    "AW": -1,
    "AW0": -1,
    "AW1": -1,
    "AW2": -1,
    "AY": -1,
    "AY0": -1,
    "AY1": -1,
    "AY2": -1,
    "EH": 1,
    "EH0": 1,
    "EH1": 1,
    "EH2": 1,
    "ER": -1,
    "ER0": -1,
    "ER1": -1,
    "ER2": -1,
    "EY": 1,
    "EY0": 1,
    "EY1": 1,
    "EY2": 1,
    "IH": 1,
    "IH0": 1,
    "IH1": 1,
    "IH2": 1,
    "IY": 1,
    "IY0": 1,
    "IY1": 1,
    "IY2": 1,
    "OW": -1,
    "OW0": -1,
    "OW1": -1,
    "OW2": -1,
    "OY": -1,
    "OY0": -1,
    "OY1": -1,
    "OY2": -1,
    "UH": -1,
    "UH0": -1,
    "UH1": -1,
    "UH2": -1,
    "UW": -1,
    "UW0": -1,
    "UW1": -1,
    "UW2": -1,
}

_back = {
    "AA": 1,
    "AA0": 1,
    "AA1": 1,
    "AA2": 1,
    "AE": -1,
    "AE0": -1,
    "AE1": -1,
    "AE2": -1,
    "AH": -1,
    "AH0": -1,
    "AH1": -1,
    "AH2": -1,
    "AO": 1,
    "AO0": 1,
    "AO1": 1,
    "AO2": 1,
    "AW": 1,
    "AW0": 1,
    "AW1": 1,
    "AW2": 1,
    "AY": 1,
    "AY0": 1,
    "AY1": 1,
    "AY2": 1,
    "EH": -1,
    "EH0": -1,
    "EH1": -1,
    "EH2": -1,
    "ER": -1,
    "ER0": -1,
    "ER1": -1,
    "ER2": -1,
    "EY": -1,
    "EY0": -1,
    "EY1": -1,
    "EY2": -1,
    "IH": -1,
    "IH0": -1,
    "IH1": -1,
    "IH2": -1,
    "IY": -1,
    "IY0": -1,
    "IY1": -1,
    "IY2": -1,
    "OW": 1,
    "OW0": 1,
    "OW1": 1,
    "OW2": 1,
    "OY": 1,
    "OY0": 1,
    "OY1": 1,
    "OY2": 1,
    "UH": 1,
    "UH0": 1,
    "UH1": 1,
    "UH2": 1,
    "UW": 1,
    "UW0": 1,
    "UW1": 1,
    "UW2": 1,
}

_high = {
    "AA": -1,
    "AA0": -1,
    "AA1": -1,
    "AA2": -1,
    "AE": -1,
    "AE0": -1,
    "AE1": -1,
    "AE2": -1,
    "AH": -1,
    "AH0": -1,
    "AH1": -1,
    "AH2": -1,
    "AO": -1,
    "AO0": -1,
    "AO1": -1,
    "AO2": -1,
    "AW": -1,
    "AW0": -1,
    "AW1": -1,
    "AW2": -1,
    "AY": -1,
    "AY0": -1,
    "AY1": -1,
    "AY2": -1,
    "EH": -1,
    "EH0": -1,
    "EH1": -1,
    "EH2": -1,
    "ER": -1,
    "ER0": -1,
    "ER1": -1,
    "ER2": -1,
    "EY": -1,
    "EY0": -1,
    "EY1": -1,
    "EY2": -1,
    "IH": 1,
    "IH0": 1,
    "IH1": 1,
    "IH2": 1,
    "IY": 1,
    "IY0": 1,
    "IY1": 1,
    "IY2": 1,
    "OW": -1,
    "OW0": -1,
    "OW1": -1,
    "OW2": -1,
    "OY": -1,
    "OY0": -1,
    "OY1": -1,
    "OY2": -1,
    "UH": 1,
    "UH0": 1,
    "UH1": 1,
    "UH2": 1,
    "UW": 1,
    "UW0": 1,
    "UW1": 1,
    "UW2": 1,
}

_low = {
    "AA": 1,
    "AA0": 1,
    "AA1": 1,
    "AA2": 1,
    "AE": 1,
    "AE0": 1,
    "AE1": 1,
    "AE2": 1,
    "AH": -1,
    "AH0": -1,
    "AH1": -1,
    "AH2": -1,
    "AO": -1,
    "AO0": -1,
    "AO1": -1,
    "AO2": -1,
    "AW": 1,
    "AW0": 1,
    "AW1": 1,
    "AW2": 1,
    "AY": 1,
    "AY0": 1,
    "AY1": 1,
    "AY2": 1,
    "EH": -1,
    "EH0": -1,
    "EH1": -1,
    "EH2": -1,
    "ER": -1,
    "ER0": -1,
    "ER1": -1,
    "ER2": -1,
    "EY": -1,
    "EY0": -1,
    "EY1": -1,
    "EY2": -1,
    "IH": -1,
    "IH0": -1,
    "IH1": -1,
    "IH2": -1,
    "IY": -1,
    "IY0": -1,
    "IY1": -1,
    "IY2": -1,
    "OW": -1,
    "OW0": -1,
    "OW1": -1,
    "OW2": -1,
    "OY": -1,
    "OY0": -1,
    "OY1": -1,
    "OY2": -1,
    "UH": -1,
    "UH0": -1,
    "UH1": -1,
    "UH2": -1,
    "UW": -1,
    "UW0": -1,
    "UW1": -1,
    "UW2": -1,
}

_round = {
    "AA": -1,
    "AA0": -1,
    "AA1": -1,
    "AA2": -1,
    "AE": -1,
    "AE0": -1,
    "AE1": -1,
    "AE2": -1,
    "AH": -1,
    "AH0": -1,
    "AH1": -1,
    "AH2": -1,
    "AO": 1,
    "AO0": 1,
    "AO1": 1,
    "AO2": 1,
    "AW": -1,
    "AW0": -1,
    "AW1": -1,
    "AW2": -1,
    "AY": -1,
    "AY0": -1,
    "AY1": -1,
    "AY2": -1,
    "EH": -1,
    "EH0": -1,
    "EH1": -1,
    "EH2": -1,
    "ER": -1,
    "ER0": -1,
    "ER1": -1,
    "ER2": -1,
    "EY": -1,
    "EY0": -1,
    "EY1": -1,
    "EY2": -1,
    "IH": -1,
    "IH0": -1,
    "IH1": -1,
    "IH2": -1,
    "IY": -1,
    "IY0": -1,
    "IY1": -1,
    "IY2": -1,
    "OW": 1,
    "OW0": 1,
    "OW1": 1,
    "OW2": 1,
    "OY": 1,
    "OY0": 1,
    "OY1": 1,
    "OY2": 1,
    "UH": 1,
    "UH0": 1,
    "UH1": 1,
    "UH2": 1,
    "UW": 1,
    "UW0": 1,
    "UW1": 1,
    "UW2": 1,
}

_rhotic = {
    "AA": -1,
    "AA0": -1,
    "AA1": -1,
    "AA2": -1,
    "AE": -1,
    "AE0": -1,
    "AE1": -1,
    "AE2": -1,
    "AH": -1,
    "AH0": -1,
    "AH1": -1,
    "AH2": -1,
    "AO": -1,
    "AO0": -1,
    "AO1": -1,
    "AO2": -1,
    "AW": -1,
    "AW0": -1,
    "AW1": -1,
    "AW2": -1,
    "AY": -1,
    "AY0": -1,
    "AY1": -1,
    "AY2": -1,
    "EH": -1,
    "EH0": -1,
    "EH1": -1,
    "EH2": -1,
    "ER": 1,
    "ER0": 1,
    "ER1": 1,
    "ER2": 1,
    "EY": -1,
    "EY0": -1,
    "EY1": -1,
    "EY2": -1,
    "IH": -1,
    "IH0": -1,
    "IH1": -1,
    "IH2": -1,
    "IY": -1,
    "IY0": -1,
    "IY1": -1,
    "IY2": -1,
    "OW": -1,
    "OW0": -1,
    "OW1": -1,
    "OW2": -1,
    "OY": -1,
    "OY0": -1,
    "OY1": -1,
    "OY2": -1,
    "UH": -1,
    "UH0": -1,
    "UH1": -1,
    "UH2": -1,
    "UW": -1,
    "UW0": -1,
    "UW1": -1,
    "UW2": -1,
}

_fronting_diphthong = {
    "AA": -1,
    "AA0": -1,
    "AA1": -1,
    "AA2": -1,
    "AE": -1,
    "AE0": -1,
    "AE1": -1,
    "AE2": -1,
    "AH": -1,
    "AH0": -1,
    "AH1": -1,
    "AH2": -1,
    "AO": -1,
    "AO0": -1,
    "AO1": -1,
    "AO2": -1,
    "AW": -1,
    "AW0": -1,
    "AW1": -1,
    "AW2": -1,
    "AY": 1,
    "AY0": 1,
    "AY1": 1,
    "AY2": 1,
    "EH": -1,
    "EH0": -1,
    "EH1": -1,
    "EH2": -1,
    "ER": -1,
    "ER0": -1,
    "ER1": -1,
    "ER2": -1,
    "EY": 1,
    "EY0": 1,
    "EY1": 1,
    "EY2": 1,
    "IH": -1,
    "IH0": -1,
    "IH1": -1,
    "IH2": -1,
    "IY": 1,
    "IY0": 1,
    "IY1": 1,
    "IY2": 1,
    "OW": -1,
    "OW0": -1,
    "OW1": -1,
    "OW2": -1,
    "OY": 1,
    "OY0": 1,
    "OY1": 1,
    "OY2": 1,
    "UH": -1,
    "UH0": -1,
    "UH1": -1,
    "UH2": -1,
    "UW": -1,
    "UW0": -1,
    "UW1": -1,
    "UW2": -1,
}

_backing_diphthong = {
    "AA": -1,
    "AA0": -1,
    "AA1": -1,
    "AA2": -1,
    "AE": -1,
    "AE0": -1,
    "AE1": -1,
    "AE2": -1,
    "AH": -1,
    "AH0": -1,
    "AH1": -1,
    "AH2": -1,
    "AO": -1,
    "AO0": -1,
    "AO1": -1,
    "AO2": -1,
    "AW": 1,
    "AW0": 1,
    "AW1": 1,
    "AW2": 1,
    "AY": -1,
    "AY0": -1,
    "AY1": -1,
    "AY2": -1,
    "EH": -1,
    "EH0": -1,
    "EH1": -1,
    "EH2": -1,
    "ER": -1,
    "ER0": -1,
    "ER1": -1,
    "ER2": -1,
    "EY": -1,
    "EY0": -1,
    "EY1": -1,
    "EY2": -1,
    "IH": -1,
    "IH0": -1,
    "IH1": -1,
    "IH2": -1,
    "IY": -1,
    "IY0": -1,
    "IY1": -1,
    "IY2": -1,
    "OW": 1,
    "OW0": 1,
    "OW1": 1,
    "OW2": 1,
    "OY": -1,
    "OY0": -1,
    "OY1": -1,
    "OY2": -1,
    "UH": -1,
    "UH0": -1,
    "UH1": -1,
    "UH2": -1,
    "UW": 1,
    "UW0": 1,
    "UW1": 1,
    "UW2": 1,
}

_stress = {
    "AA": 1,
    "AA0": -1,
    "AA1": 1,
    "AA2": -1,
    "AE": 1,
    "AE0": -1,
    "AE1": 1,
    "AE2": -1,
    "AH": 1,
    "AH0": -1,
    "AH1": 1,
    "AH2": -1,
    "AO": 1,
    "AO0": -1,
    "AO1": 1,
    "AO2": -1,
    "AW": 1,
    "AW0": -1,
    "AW1": 1,
    "AW2": -1,
    "AY": 1,
    "AY0": -1,
    "AY1": 1,
    "AY2": -1,
    "EH": 1,
    "EH0": -1,
    "EH1": 1,
    "EH2": -1,
    "ER": 1,
    "ER0": -1,
    "ER1": 1,
    "ER2": -1,
    "EY": 1,
    "EY0": -1,
    "EY1": 1,
    "EY2": -1,
    "IH": 1,
    "IH0": -1,
    "IH1": 1,
    "IH2": -1,
    "IY": 1,
    "IY0": -1,
    "IY1": 1,
    "IY2": -1,
    "OW": 1,
    "OW0": -1,
    "OW1": 1,
    "OW2": -1,
    "OY": 1,
    "OY0": -1,
    "OY1": 1,
    "OY2": -1,
    "UH": 1,
    "UH0": -1,
    "UH1": 1,
    "UH2": -1,
    "UW": 1,
    "UW0": -1,
    "UW1": 1,
    "UW2": -1,
}

In [ ]:
_cons_set = {
    "B",
    "CH",
    "D",
    "DH",
    "F",
    "G",
    "HH",
    "JH",
    "K",
    "L",
    "M",
    "N",
    "NG",
    "P",
    "R",
    "S",
    "SH",
    "T",
    "TH",
    "V",
    "W",
    "Y",
    "Z",
    "ZH",
}